# Structure from Motion

### The Projective space

Points in $ \mathbb{P}^2 $ are represented as

$$ \mathbf{x} = \begin{bmatrix}
x & y & w
\end{bmatrix}^T
$$

The equivalent point in $ \mathbb{R}^2 $ is

$$ \mathbf{x} = \begin{bmatrix}
x / w & y / w
\end{bmatrix}^T
$$

For points with $w \neq 0$. Points with $w=0$ is called points as infinity.

Lines are represented as

$$
\mathbf{x}^T\mathbf{l} = 0
$$

Where 

$$ \mathbf{l} = \begin{bmatrix}
a & b & c
\end{bmatrix}^T
$$


### Null space

Finding any non-zero vector $\mathbf{x}$ satifying the equation

$$
A\mathbf{x} = \mathbf{0}
$$

The nullspace is only determined up to a scale.

In [1]:
import numpy as np
from numpy.linalg import svd

def nullspace(A, atol=1e-13, rtol=0):
    A = np.atleast_2d(A)
    u, s, vh = svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

A = np.array([[1, 0],
              [0, 1]])
print(nullspace(A))

A = np.array([[1, 0],
              [0, 0]])
n = nullspace(A)
print(n)
print(A @ n)
k = 17
print(A @ (k * n))


### Projective camera

A projective camera $\texttt{P}$ transforms a world point $\mathbf{X}$ into a image point $\mathbf{x}.$

$$\mathbf{x} = \texttt{P}\mathbf{X}$$

Where $\texttt{P}$ is a 3x4 matrix and the world point $\mathbf{X}$ is a 4-vector while the imgage point $\mathbf{x}$ is a 3-vector.

In [2]:
P = np.array([
    [20,  0, 0, -10],
    [ 0, 20, 0, -10],
    [ 0,  0, 1, 0],
])
X = [3, 4, 10, 1]
x = P @ X
# todo plot this
print(x / x[2])

The camera matrix $\texttt{P}$ can be decomposed into the calibration matrix $\texttt{K}$ and a rotation and translation pair like so

$$
\texttt{P} = \texttt{K}
\begin{bmatrix}
\texttt{R} & \mathbf{t}
\end{bmatrix}
$$

Sometimes the camera centre $\tilde{C}$ is needed expicitly

$$
\mathbf{t} = -\texttt{R}\tilde{C}
$$

In [3]:
K = np.array([
    [10, 0, 50],
    [0, 10, 50],
    [0, 0, 1]
])
R = np.eye(3)
t = np.array([10, 10, 0])
P = K @ np.hstack([R, t[:, None]])
print(P)

### Calibration matrix

In [4]:
from ipywidgets import interact
from ipycplot import Plot3D, rectangle
import numgl
from numpy.linalg import inv

width, height = 320, 200

n = 10
x, y = np.meshgrid(np.linspace(0, width, n), np.linspace(0, height, n))
w = np.ones(x.shape)
# points in the image plane
p1 = np.stack([x, y, w], axis=-1).reshape((-1, 3))

plot = Plot3D()
plot.scale = np.array([200, 200, 1])
target = np.array([width/2, height/2, 0])
plot.camera.lookat(target=target)
def update_focal(f, ay, px, py):
    K = np.array([
        [f, 0, px],
        [0, f, py],
        [0, 0, 1],
    ])
    normals = (inv(K) @ p1.T).T
    p2 = p1 + numgl.normalized(normals.T, axis=0).T * 50
    lines = np.hstack([p1, p2]).reshape(-1, 3)
    
    r = 600
    eye = np.array([r * np.sin(ay), 0, r * np.cos(ay)])
    plot.camera.lookat(eye=eye + target)
    
    plot.canvas.clear()
    plot.lines(lines)
    plot.lines(rectangle(width, height))

interact(update_focal, f=(50, 1000), ay=(-np.pi/2, np.pi/2), px=(0, width), py=(0, height))
plot.show()

interactive(children=(IntSlider(value=525, description='f', max=1000, min=50), FloatSlider(value=0.0, descript…

RoughCanvas()

### Epipolar Geometry
The geometry of two cameras depicting the same scene. 
* The _epipolar point_ is the image of the camera center of the other image. It may lie outside the visible image.
* The _epipolar line_ is the line between the two camera centers.


In [183]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numgl
from ipywidgets import interact
from ipycplot import Plot3D, rectangle, transform


width, height = 320, 200

# world points
n = 10
X = np.random.normal(size=(n, 3), scale=100)

def rt(pose):
    R, C = pose[0:3, 0:3], pose[0:3, 3][:, None]
    t = -R.T @ C
    return R, t

plot = Plot3D()
plot.scale = np.array([80, 80, 1])

f = 350
K = np.array([
    [f, 0, width/2],
    [0, f, height/2],
    [0, 0, 1],
])

left_pose =  numgl.translate((-190,    0, 500)) @ numgl.roty(2.8)
right_pose = numgl.translate(( 330, -100, 500)) @ numgl.roty(3.7)
P1 = K @ np.hstack(rt(left_pose))
P2 = K @ np.hstack(rt(right_pose))

def mask_points(x, width, height):
    mask = (x[:, 0] >= 0) * (x[:, 0] < width) * (x[:, 1] >= 0) * (x[:, 1] < height)
    return x[mask]

def update(ay):
    center = np.array([0, 150, 0])
    target = center
    r = 1100
    eye = center + np.array([r * np.sin(ay), 0, r * np.cos(ay)]) + np.array([0, -300, 0])
    plot.camera.lookat(eye=eye, target=target)

    plot.canvas.clear()

    # plot world points
    plot.circles(X)    
    
    # project points in left camera
    
    x1 = transform(P1, X)  # transform world points using camera
    x1 = mask_points(x1, width, height)  # mask out points outside screen
    x1 = np.hstack([x1, np.zeros((x1.shape[0], 1))])  # add z=0 coordinate
    plot.circles(transform(left_pose, x1))  # plot projected points for camera
    plot.lines(transform(left_pose, rectangle(width, height)))

    # project points in right camera
    x2 = transform(P2, X)  # transform world points using camera
    x2 = mask_points(x2, width, height)  # mask out points outside screen
    x2 = np.hstack([x2, np.zeros((x2.shape[0], 1))])  # add z=0 coordinate
    plot.circles(transform(right_pose, x2))  # plot projected points for camera
    plot.lines(transform(right_pose, rectangle(width, height)))
    
interact(update, ay=(-np.pi/2, np.pi/2))

plot.show()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


interactive(children=(FloatSlider(value=0.0, description='ay', max=1.5707963267948966, min=-1.5707963267948966…

RoughCanvas()

### The Fundamental Matrix
The fundamental matrix maps points in the left image to lines in the right image.

$$
\mathbf{l'} = \texttt{F}\mathbf{x}
$$

A point in the left image $\mathbf{x}$ corresponding to the same world point $\mathbf{X}$ will lie on the epipolar line $\mathbf{l'}$ in the right image, so we have

$$
\mathbf{x'}^T\texttt{F}\mathbf{x} = \mathbf{0}
$$

It can be directly computed from two cameras like so

$$
\texttt{F} =
\begin{bmatrix}
\mathbf{e}'
\end{bmatrix}_\times 
\texttt{P}'\texttt{P}^+
$$

In [6]:
from numpy.linalg import pinv

def skew(x):
    return np.array([
        [    0, -x[2],  x[1]],
        [ x[2],     0, -x[0]],
        [-x[1],  x[0],    0]])

def from_cameras(P1, P2):
    C1 = nullspace(P1).flatten()  # camera center
    e2 = P2 @ C1  # epipole
    return skew(e2) @ P2 @ pinv(P1)

F = from_cameras(P1, P2)
print(F)

[[-1.46447971e-01 -4.81326969e-01  1.12239257e+02]
 [-2.81864098e-01 -2.55593651e-14  3.38446758e+02]
 [ 1.09432008e+01 -1.82613145e+02 -3.72723058e+04]]


In [7]:
print(nullspace(F))
print(nullspace(F.T))

[[ 9.93997891e-01]
 [-1.09396108e-01]
 [ 8.27818002e-04]]
[[ 0.84917653]
 [-0.52810436]
 [-0.00223824]]


In [8]:
def rank(A, tol=1e-13):
    _, s, _ = svd(A)
    return np.sum(s > tol)

rank(F)

2

In [13]:
from numpy import cross
from ipycplot import Plot2D, dehomogenize
def from_implicit(l, width, height):
    """Create a line segment from the intersection of an implicit line with a rectangle"""
    left = np.array([1, 0, 0])  # x = 0
    right = np.array([1, 0, -(width - 1)])  # x = width
    top = np.array([0, 1, 0])  # y = 0
    bottom = np.array([0, 1, -(height - 1)])  # y = height
    intersections = np.array([
        cross(l, left), cross(l, right), cross(l, top), cross(l, bottom),
    ])
    masked = mask_points(dehomogenize(intersections.T).T, width, height)
    return masked

x1 = np.hstack([np.random.rand(2) * np.array([width, height]), 1])  # random image point
print(x1)

plot = Plot2D()
plot.lines(from_implicit(F@x1, width, height))
plot.show()

[252.3643734   99.57053992   1.        ]


RoughCanvas()

### Fundamental Matrix from correspondances
Using $\mathbf{x'}^T\texttt{F}\mathbf{x} = \mathbf{0}$ we can compute $\texttt{F}$ from image correspondances alone. For a single correspondance $\mathbf{x} \leftrightarrow \mathbf{x}'$ where $\mathbf{x} = \left [ x \,  y \, 1  \right ]^T$ and $\mathbf{x}' = \left [ x '\,  y' \, 1  \right ]^T$ we can expand the inner products like so

$$
x'xf_{11} + x'yf_{12} + xf_{13} + y'xf_{21} + y'yf_{22} + y'f_{23} + xf_{31} + yf_{32} + f_{33} = 0
$$

If we introduce a vector $ \mathbf{f} = \left [ f_{11}, f_{12}, f_{13}, f_{21}, f_{22}, f_{23}, f_{31}, f_{32}, f_{33} \right ]^T $ it can be written as 

$$
\left [ x'x, x'y, x', y'x, y'y, y', x, y, 1  \right ] \mathbf{f} = \mathbf{0}
$$

Given several image correspondances we can stack these like so
$$
A\mathbf{f} =
\begin{bmatrix}
x_1'x_1 & x_1'y_1 & x_1' & y_1'x_1 & y_1'y_1 & y_1' & x_1 & y_1 & 1 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
x_n'x_n & x_n'y_n & x_n' & y_n'x_n & y_n'y_n & y_n' & x_n & y_n & 1 \\
\end{bmatrix} \mathbf{f}  = \mathbf{0}
$$



In [145]:
from numpy.linalg import norm
#from ipycplot import transform

def normalizing_transform(p):
    """Isotropic point normalization. Returns a transform that transform to mean 0, 0 and mean norm=sqrt(2)"""
    d = p.shape[1]
    c = np.mean(p, axis=0)
    s = np.mean(norm(p - c[None:], axis=-1))
    T = np.diag(np.append(np.repeat(s / np.sqrt(2), d), 1))
    T[:d, d] = c
    return inv(T)

x1 = transform(P1, X)  # transform world points using camera
x2 = transform(P2, X)  # transform world points using camera

# compute F
def from_correspondances(x1, x2):
    # compute normalizing transforms
    T1 = normalizing_transform(x1)
    T2 = normalizing_transform(x2)
    nx1 = transform(T1, x1)
    nx2 = transform(T2, x2)
    A = np.vstack([(x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1, 1) for (x1, y1), (x2, y2) in zip(nx1, nx2)])
    f = nullspace(A)
    F = f.reshape((3, 3))    
    
    # force rank 2
    u, s, vh = svd(F)
    s[-1] = 0
    F = u @ np.diag(s) @ vh
    # denormalize F matrix
    return T2.T @ F @ T1

F = from_correspondances(x1, x2)
print(F)


[[ 7.48113124e-06  2.45880513e-05 -5.73361723e-03]
 [ 1.43987130e-05 -7.93073622e-21 -1.72891751e-02]
 [-5.59021205e-04  9.32858880e-03  1.90401417e+00]]


### Triangulation

From the projection of world points $\mathbf{X}$ we have
$$
\mathbf{x} = \texttt{P}\mathbf{X} \\
\mathbf{x}' = \texttt{P}'\mathbf{X}
$$

The scale factor can be eliminated by using cross product, e.g. for the left image $\mathbf{x} \times (\texttt{P}\mathbf{X}) = \mathbf{0}$

In [208]:
def triangulate(P1, P2, x1, x2):
    Xs = []
    for (x1, y1), (x2, y2) in zip(x1, x2):
        A = np.array([
            x1 * P1[2, :] - P1[0, :],
            y1 * P1[2, :] - P1[1, :],
            x2 * P2[2, :] - P2[0, :],
            y2 * P2[2, :] - P2[1, :],
        ])
        u, s, vh = svd(A)
        Xs.append(vh[-1])
    return np.array(Xs)
        
Xr = triangulate(P1, P2, x1, x2)  # triangulate points
print(dehomogenize(P1 @ Xr.T).T)  # project back into image
print(x1)  # original image points

[[157.53511931 111.88802676]
 [229.26891996  92.24687738]
 [ 69.26736717 223.54866781]
 [259.31181305  95.65058541]
 [179.00201482  35.46208413]
 [129.59500509 123.99361912]
 [187.33473031  92.70056335]
 [155.79554764 135.7857864 ]
 [165.55440464   4.09336727]
 [176.26142953 164.18781853]]
[[157.53511931 111.88802676]
 [229.26891996  92.24687738]
 [ 69.26736717 223.54866781]
 [259.31181305  95.65058541]
 [179.00201482  35.46208413]
 [129.59500509 123.99361912]
 [187.33473031  92.70056335]
 [155.79554764 135.7857864 ]
 [165.55440464   4.09336727]
 [176.26142953 164.18781853]]


### Extracting cameras

In [253]:
from numpy.linalg import det

def depth(P, X):
    """Computes the depth of a world point X given a camera P"""
    Xh = homogenize(X[:, None])
    M = P[:3, :3]
    w = (P @ Xh).flatten()[2]
    W = Xh.flatten()[3]
    return np.sign(det(M)) * w / (W * norm(M[-1]))  # or M[:, 2] ?

def extract_cameras(K1, K2, F):
    W = np.array([
        [0, -1, 0],
        [1,  0, 0],
        [0,  0, 1],
    ])  # orthonogal (W.T@W = W@W.T = I)
    Z = np.array([
        [ 0, 1, 0],
        [-1, 0, 0],
        [ 0, 0, 0],
    ])  # skew-symetric (-Z = Z.T)
    E = K2.T @ F @ K1
    U, s, Vh = svd(E)
    u3 = u.T[-1]
    
    alternatives = [
        (U @ W @ Vh, u3),
        (U @ W @ Vh, -u3),
        (U @ W.T @ Vh.T, u3),
        (U @ W.T @ Vh.T, -u3),
    ]
    return alternatives

extract_cameras(K, K, F)
depth(P1, X[6])

511.05641185863163

# Images
